# Overview



## Prerequisites 
<br>
<p>Add the following to the DEFAULT profile to your \.oci\config file:
    </p>
    
```sh
target_profile = <Your Profile>
```

<p>The value in your target profile will be your working profile.  Under this profile, add the following:</p>

```sh
auth_profile = <Name of your security profile>
target_compartment = <Compartment where you will create Bastion Sessions>
```


## Assumptions

##### Windows environments only can be used

# Prerequisites

## Imports and Initializations

In [ ]:
import oci
from pyUtils.token_signer import token_signer
import pprint, json
import datetime
import pytz
from dateutil.relativedelta import relativedelta
from IPython.display import display, Math, HTML, Markdown
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 70em; }</style>"))

# Read "target_profile" from .oci/config file
try:
    tenancyProfile = oci.config.from_file()['target_profile']
except:
    print('A Target Profile not found in config file.  Using Default Profile')
    tenancyProfile = 'DEFAULT'

tenancyConfig = oci.config.from_file(profile_name=tenancyProfile)
bastion_client = oci.bastion.BastionClient(
                        {'region':oci.config.from_file(profile_name=tenancyProfile)['region']},
                        signer=token_signer(tenancyConfig['auth_profile'])  )

def create_session (session_type,sessionName,resourcePort,targetResourceId,targetIP):
    create_session_response = bastion_client.create_session(
        create_session_details=oci.bastion.models.CreateSessionDetails(
            bastion_id=bastionId,
            target_resource_details=oci.bastion.models.CreateManagedSshSessionTargetResourceDetails(
                session_type=session_type,
                target_resource_operating_system_user_name="opc",
                target_resource_id= targetResourceId ,
                target_resource_port=resourcePort,
                target_resource_private_ip_address=targetIP),
            key_details=keyDetails,
            display_name=sessionName,
            key_type="PUB",
            session_ttl_in_seconds=10800)
        ) 
    return create_session_response.__dict__['data']


## Required settings

In [ ]:
sessionNamePrefix = 'My_Bastion_'
forwardPort = 2030
# Required Settings
bastionId=  tenancyConfig['target_bastion']
targetResourceId = tenancyConfig['target_bastion_resource']
targetIP = tenancyConfig['target_bastion_resource_ip']
publicKeyLocation = tenancyConfig['bastion_public_key_location']
privateKeyLocation = tenancyConfig['bastion_private_key_location']
preferred_timezone = tenancyConfig['preferred_timezone']
with open(publicKeyLocation, mode='r') as public_file:
    key_data = public_file.read()
sshKey=key_data
keyDetails = oci.bastion.models.PublicKeyDetails(public_key_content=sshKey) 

# Bastion

## List Bastion Sessions and SSH Commands

In [ ]:
bastion_client = oci.bastion.BastionClient(
                        {'region':oci.config.from_file(profile_name=tenancyProfile)['region']},
                        signer=token_signer(tenancyConfig['auth_profile'])  )

list_sessions_response = bastion_client.list_sessions( bastion_id=bastionId )
sessions = list_sessions_response.__dict__['data']
print('\nMy Sessions\n--------\n')

for session in sessions:
    if 'ACTIVE' in session.lifecycle_state and sessionNamePrefix in session.display_name:
#             sessionNamePrefix in session.target_resource_details.target_resource_display_name:
        print('Session Name: ' ,session.display_name)
        print('    Session Type: ' ,session.target_resource_details.session_type)
        print('    Session Created: ' ,session.time_created.astimezone(pytz.timezone(preferred_timezone)).strftime("%m/%d/%Y %H:%M:%S %Z")   )
        timeout = session.time_created  + relativedelta(seconds=session.session_ttl_in_seconds) 
        print('    Session Timeout: ' ,  timeout.astimezone(pytz.timezone(preferred_timezone)).strftime("%m/%d/%Y %H:%M:%S %Z")  )
        print()
        devSessionData = bastion_client.get_session(session.id)
        devSessionSSH = devSessionData.__dict__['data'].ssh_metadata['command']
#         print('SSH Command: ', devSessionSSH)
        if 'PORT' in str(session.target_resource_details.session_type):
            devSessionSSH = devSessionSSH + ' -v'
        devSessionSSH = devSessionSSH.replace('<privateKey>',privateKeyLocation)
        devSessionSSH = devSessionSSH.replace('<localPort>',str(forwardPort))
        sshCmd = """
```os
SSHCMD
```
"""
        print('    SSH Command: ' )
        display(Markdown(sshCmd.replace('SSHCMD',devSessionSSH  )     ))
#         print('    SSH Command: \n\n', devSessionSSH.replace('<localPort>',str(forwardPort)))
        print('\n    -------\n')

print('\n\n')

## Create Bastion Session

### Create Managed SSH Session

In [ ]:
bastion_client = oci.bastion.BastionClient(
                        {'region':oci.config.from_file(profile_name=tenancyProfile)['region']},
                        signer=token_signer(tenancyConfig['auth_profile'])  )

session_type = "MANAGED_SSH"
resourcePort = 22

managedSession = create_session(session_type,sessionNamePrefix+session_type,
                             resourcePort,targetResourceId, targetIP  )
print('Managed Session -  ', managedSession.display_name + ' - being created in in Bastion: ' + managedSession.bastion_name  )
print('Session being create for ' + managedSession.target_resource_details.target_resource_display_name 
     + ' with IP Address: ' +   managedSession.target_resource_details.target_resource_private_ip_address
     + ' using Port ' + str(managedSession.target_resource_details.target_resource_port))
print('\nRun List of Bastion Session to verify when Session is completed')
print('Copy and paste ssh command to your MobaXterm (or whatever you are using)')
# pprint.pprint(managedSession)

### Create Port Forwarding Session

In [ ]:
session_type = "PORT_FORWARDING"
sessionName = 'DevInst2_Port_Forwarding'
resourcePort = 22

portForwardSession = create_session(session_type,sessionNamePrefix+session_type,resourcePort, targetResourceId, targetIP  )

print('Port Forwarding Session -  ', portForwardSession.display_name + ' - being created in in Bastion: ' + portForwardSession.bastion_name  )
print('Session being create for ' + portForwardSession.target_resource_details.target_resource_display_name 
     + ' with IP Address: ' +   portForwardSession.target_resource_details.target_resource_private_ip_address
     + ' using Port ' + str(portForwardSession.target_resource_details.target_resource_port))
print('\nRun List of Bastion Session to verify when Session is completed')
print('Copy and paste ssh command to your MobaXterm (or whatever you are using)')
# pprint.pprint(portForwardSession)